In [1]:
from neu import NeuMF
import pickle
import pandas as pd
import torch
from torch import nn
from sklearn.preprocessing import LabelEncoder
from concurrent.futures import ProcessPoolExecutor
import numpy as np
from operator import itemgetter
THREADS = 16
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import time
import torch.nn.functional as F
loss_func = torch.nn.MSELoss()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
mlp_factors = 8
gmf_factors = 32 
layers = [64,32,16,8]

In [2]:
def binary_predictions(true_ratings, predicted_ratings):
    assert len(true_ratings) == len(predicted_ratings)
    binary_true_ratings = []
    binary_predicted_ratings = []

    for i in range(len(true_ratings)):
        if true_ratings[i] >= 3:
            binary_true_ratings.append(1)
        else:
            binary_true_ratings.append(0)

        if predicted_ratings[i] >= 3:
            binary_predicted_ratings.append(1)
        else:
            binary_predicted_ratings.append(0)

    return precision_score(binary_true_ratings, binary_predicted_ratings), recall_score(binary_true_ratings, binary_predicted_ratings), f1_score(binary_true_ratings, binary_predicted_ratings)

In [3]:
def arg_accuracy_int(ratings, predictions):
    ratings = ratings.cpu().detach().numpy()
    predictions = predictions.cpu().detach().numpy()
    total_nr = len(ratings)
    total_pred = 0
    for i in range(total_nr):
        (true_rating, pred_rating) = ratings[i], predictions[i]
        if round(pred_rating) >= int(true_rating)-1 and round(pred_rating) <= int(true_rating)+1:
            total_pred += 1

    return float(total_pred)/total_nr


def round_of_rating(number):
    return round(number * 2) / 2

In [4]:
def RMSE(data, model):
    users_index = data.iloc[:, 0].values
    users = torch.LongTensor(users_index).to(DEVICE)
    items_index = data.iloc[:, 1].values
    items = torch.LongTensor(items_index).to(DEVICE)
    rating = torch.FloatTensor(data.iloc[:, 5].values).to(DEVICE)
    prediction= model(users, items)
    rmse = loss_func(prediction, rating)
    mae = torch.nn.L1Loss()(prediction, rating)
    
    p,r,f = binary_predictions(rating, prediction)
    accuracy = arg_accuracy_int(rating,prediction)
    return rmse ** 0.5,mae,p,r,f, accuracy

In [5]:
import json
trainset = pd.read_csv('train.csv')
testset = pd.read_csv('test.csv')
trainset['user_rating'] = (trainset['user_rating'] + 1) * 2 + 1
testset['user_rating'] = (testset['user_rating'] + 1) * 2 + 1
def traite_train_test(df):
    df['actors'] = df['actors'].apply(lambda x: json.loads(x))
    df['director'] = df['director'].apply(lambda x: json.loads(x))
    df['genre'] = df['genre'].apply(lambda x: json.loads(x))
    return df
trainset = traite_train_test(trainset)
testset = traite_train_test(testset)
hehe_test = trainset.copy()
df_empty = testset.copy()
df_empty['user_id'] = df_empty['user_id'].astype('int')
df_empty['user_rating'] = df_empty['user_rating'].astype('float')
df_empty['movie'] = df_empty['movie'].astype('int')
hehe_test.index = range(len(hehe_test))
df_empty.index = range(len(df_empty))

In [6]:
hehe = pd.concat([hehe_test,df_empty])

In [7]:
def train(lr, mlp_factors,gmf_factors, layers, reg, batch_size, num_epochs, train, test):
    model = NeuMF(n_users, n_items, mlp_factors,gmf_factors, layers).to(DEVICE)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr,weight_decay=reg)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10, threshold_mode='abs',threshold = 0.005)
    for epoch in range(num_epochs):
        model.train()
        t1 = time.time()
        num_example = len(train)
        indices = list(range(num_example))
        for i in tqdm(range(0, num_example, batch_size)):
            optimizer.zero_grad()
            indexs = indices[i:min(i+batch_size, num_example)]
            users_index = train.iloc[:, 0].loc[indexs].values
            users = torch.LongTensor(users_index).to(DEVICE)
            items_index = train.iloc[:, 1].loc[indexs].values
            items = torch.LongTensor(items_index).to(DEVICE)
            
           
            rating = torch.FloatTensor(
                train.iloc[:, 5].loc[indexs].values).to(DEVICE)
            prediction = model(
                users, items)

            err = loss_func(prediction, rating) 
            err.backward()
            optimizer.step()
        t2 = time.time()
        #rmse, mae = RMSE(test, model)
        
        
        rmse, mae, p, r, f, accuracy = RMSE(testset,model)
        scheduler.step(rmse)
        print("Learning rate: ", lr, "Regulation: ", reg,"Bath_size:",batch_size)
        print("RMSE: ", rmse, "MAE: ", mae)
        print("Accuracy: ", accuracy, "Precision: ", p, "Recall: ", r, "F1 score: ", f)
    
    return model

In [8]:
n_users = len(hehe['user_id'].value_counts())
n_items = len(hehe['movie'].value_counts())

In [9]:
from tqdm import tqdm
lr = 0.002
reg = 1e-4
batch_size = 128
num_epochs = 5
model_neumf = train(lr, mlp_factors,gmf_factors, layers, reg, batch_size, num_epochs, hehe_test,df_empty)

  0%|          | 6/1207 [00:00<00:21, 55.71it/s]

Learning rate:  0.002 Regulation:  0.0001 Bath_size: 128
RMSE:  tensor(1.0072, grad_fn=<PowBackward0>) MAE:  tensor(0.8069, grad_fn=<L1LossBackward>)
Accuracy:  0.8559405427801947 Precision:  0.8283694211224039 Recall:  0.9034507132759285 F1 score:  0.8642825308518649


  0%|          | 6/1207 [00:00<00:22, 54.23it/s]

Learning rate:  0.002 Regulation:  0.0001 Bath_size: 128
RMSE:  tensor(0.9469, grad_fn=<PowBackward0>) MAE:  tensor(0.7505, grad_fn=<L1LossBackward>)
Accuracy:  0.8826911124922312 Precision:  0.8702795403875836 Recall:  0.8152229790515358 F1 score:  0.8418520546127176


  0%|          | 6/1207 [00:00<00:24, 49.61it/s]

Learning rate:  0.002 Regulation:  0.0001 Bath_size: 128
RMSE:  tensor(0.9307, grad_fn=<PowBackward0>) MAE:  tensor(0.7354, grad_fn=<L1LossBackward>)
Accuracy:  0.8911850010358401 Precision:  0.8801288632421143 Recall:  0.7987726513301632 F1 score:  0.8374795775715417


  0%|          | 6/1207 [00:00<00:22, 52.44it/s]

Learning rate:  0.002 Regulation:  0.0001 Bath_size: 128
RMSE:  tensor(0.9289, grad_fn=<PowBackward0>) MAE:  tensor(0.7333, grad_fn=<L1LossBackward>)
Accuracy:  0.8923244251087632 Precision:  0.8806436716280062 Recall:  0.7965235830870068 F1 score:  0.8364740615773935


100%|██████████| 1207/1207 [00:27<00:00, 44.17it/s]


Learning rate:  0.002 Regulation:  0.0001 Bath_size: 128
RMSE:  tensor(0.9287, grad_fn=<PowBackward0>) MAE:  tensor(0.7327, grad_fn=<L1LossBackward>)
Accuracy:  0.8919100890822457 Precision:  0.8805127476741709 Recall:  0.7967163603649916 F1 score:  0.8365212697770131


In [ ]:
state = { 'model': model_neumf.state_dict()}   
torch.save(state, 'model_neumf.pkl')
                    

In [ ]:
from neu import MF
mf_factors = 16

In [ ]:
def train(lr,mf_factors , reg, batch_size, num_epochs, train, test):
    model = MF(n_users, n_items,mf_factors ).to(DEVICE)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr,weight_decay=reg)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10, threshold_mode='abs',threshold = 0.005)
    for epoch in range(num_epochs):
        model.train()
        t1 = time.time()
        num_example = len(train)
        indices = list(range(num_example))
        for i in tqdm(range(0, num_example, batch_size)):
            optimizer.zero_grad()
            indexs = indices[i:min(i+batch_size, num_example)]
            users_index = train.iloc[:, 0].loc[indexs].values
            users = torch.LongTensor(users_index).to(DEVICE)
            items_index = train.iloc[:, 1].loc[indexs].values
            items = torch.LongTensor(items_index).to(DEVICE)
            
           
            rating = torch.FloatTensor(
                train.iloc[:, 5].loc[indexs].values).to(DEVICE)
            prediction = model(
                users, items)

            err = loss_func(prediction, rating) 
            err.backward()
            optimizer.step()
        t2 = time.time()
        #rmse, mae = RMSE(test, model)
        
        
        rmse, mae, p, r, f, accuracy = RMSE(testset,model)
        scheduler.step(rmse)
        print("Learning rate: ", lr, "Regulation: ", reg,"Bath_size:",batch_size)
        print("RMSE: ", rmse, "MAE: ", mae)
        print("Accuracy: ", accuracy, "Precision: ", p, "Recall: ", r, "F1 score: ", f)
    
    return model

In [ ]:
model_mf = train(lr,mf_factors , reg, batch_size, num_epochs, hehe_test,df_empty)

In [71]:
state = { 'model': model_mf.state_dict()}   
torch.save(state, 'model_mf.pkl')
                    